In [6]:
!pip install lightgbm


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached lightgbm-4.1.0-py3-none-win_amd64.whl (1.3 MB)


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve,auc, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import lightgbm as lgb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

## Read Data

In [2]:
train_df = pd.read_csv('label_df_train.csv')
test_df = pd.read_csv('label_df_valid.csv')

In [3]:
train_df.shape

(97040, 115)

In [4]:
train_df = train_df.drop(columns = ['transcript_id', 'transcript_position','gene_id','five_mer'])

In [5]:
X_train, y_train = train_df[[i for i in train_df.columns if i!='label']],train_df['label']

## Hyperparameter Tuning

In [22]:
#scaler = StandardScaler()
smote = SMOTE()
#mlp_classifier = MLPClassifier(verbose = 2)
lgb_classifier = lgb.LGBMClassifier()
pipeline = Pipeline([('smote', smote),('lgb', lgb_classifier)])

In [72]:
params_grid = {
    "smote__sampling_strategy": [0,0.25,0.33,0.5,1],
    "lgb__num_leaves":[63,70,80,90,100,110,127],
    "lgb__min_data_in_leaf":[5,10,20,40,80]
}

In [75]:
clf = RandomizedSearchCV(pipeline, param_distributions = params_grid, scoring = ["average_precision","roc_auc"], n_jobs = -2, verbose = 2, refit = False, n_iter = 50)

In [76]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python39\lib\site-packages\imblearn\pipeline.py", line 293, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "c:\Python39\lib\site-packages\imblearn\pipeline.py", line 250, in _fit
    X, y, fitted_transformer = fit_resample_one_cached(
  File "c:\Python39\lib\site-packages\joblib\memory.py", line 349, in __c

RandomizedSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('smote', SMOTE()),
                                             ('lgb', LGBMClassifier())]),
                   n_iter=50, n_jobs=-2,
                   param_distributions={'lgb__min_data_in_leaf': [5, 10, 20, 40,
                                                                  80],
                                        'lgb__num_leaves': [63, 70, 80, 90, 100,
                                                            110, 27],
                                        'smote__sampling_strategy': [0, 0.25,
                                                                     0.33, 0.5,
                                                                     1]},
                   refit=False, scoring=['average_precision', 'roc_auc'],
                   verbose=2)

In [77]:
df_results = pd.DataFrame(clf.cv_results_)

In [6]:
#scaler = StandardScaler()
smote = SMOTE()
#mlp_classifier = MLPClassifier(verbose = 2)
xgb_classifier = XGBClassifier()
pipeline = Pipeline([('smote', smote),('xgb', xgb_classifier)])

In [7]:
params_grid = {
    "smote__sampling_strategy": [0,0.25,0.33,0.5,1],
    "xgb__max_depth":[3,6,9,12,15],
    "xgb__min_child_weight":[1,2,4,8,16],
    "xgb__gamma":[0,10,20,50,100]
}

In [10]:
clf = RandomizedSearchCV(pipeline, param_distributions = params_grid, scoring = ["average_precision","roc_auc"], n_jobs = -2, verbose = 2, refit = False, n_iter = 25)

In [11]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


c:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python39\lib\site-packages\imblearn\pipeline.py", line 293, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "c:\Python39\lib\site-packages\imblearn\pipeline.py", line 250, in _fit
    X, y, fitted_transformer = fit_resample_one_cached(
  File "c:\Python39\lib\site-packages\joblib\memory.py", line 349, in __ca

RandomizedSearchCV(estimator=Pipeline(steps=[('smote', SMOTE()),
                                             ('xgb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            device=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            grow_policy=None,
                                                            importance_type=None,
                                                            intera...
                                                            monotone_constraints=None,
                                                            multi_strategy=None,
                                                            n_estimators=None,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            random_state=None, ...))]),
                   n_iter=25, n_jobs=-2,
                   param_distributions={'smote__sampling_strategy': [0, 0.25,
                                                                     0.33, 0.5,
                                                                     1],
                                        'xgb__gamma': [0, 10, 20, 50, 100],
                                        'xgb__max_depth': [3, 6, 9, 12, 15],
                                        'xgb__min_child_weight': [1, 2, 4, 8,
                                                                  16]},
                   refit=False, scoring=['average_precision', 'roc_auc'],
                   verbose=2)

In [13]:
df_results2 = pd.DataFrame(clf.cv_results_)

## Best Results

In [79]:
df_results[df_results['rank_test_average_precision']==1][['mean_test_roc_auc','mean_test_average_precision']]

,mean_test_roc_auc,mean_test_average_precision
37,0.917786,0.468722


In [15]:
df_results2[df_results2['rank_test_average_precision']==1][['mean_test_roc_auc','mean_test_average_precision']]

,mean_test_roc_auc,mean_test_average_precision
4,0.921458,0.490974


## Best Parameters

In [80]:
df_results[df_results['rank_test_average_precision']==1][['param_smote__sampling_strategy', 'param_lgb__num_leaves',
       'param_lgb__min_data_in_leaf']]

,param_smote__sampling_strategy,param_lgb__num_leaves,param_lgb__min_data_in_leaf
37,0.25,110,40


In [19]:
df_results2[df_results2['rank_test_average_precision']==1][['param_xgb__min_child_weight', 'param_xgb__max_depth',
       'param_xgb__gamma', 'param_smote__sampling_strategy']]

,param_xgb__min_child_weight,param_xgb__max_depth,param_xgb__gamma,param_smote__sampling_strategy
4,2,15,0,0.25


In [46]:
#Save results
df_results.to_csv('hyperparameter_results.csv')

## Refitting

In [12]:
lgb_scaler = StandardScaler()
lgb_smote = SMOTE(sampling_strategy=0.25)
#new_mlp_classifier = MLPClassifier(verbose = 2, hidden_layer_sizes=(16,16),learning_rate_init=0.001, activation="relu", alpha  = 0.0001, max_iter = 15)
lgb_classifier = lgb.LGBMClassifier(num_leaves=110,min_data_in_leaf = 40)
lgb_pipeline = Pipeline([('scaler', lgb_scaler),('smote', lgb_smote),('mlp', lgb_classifier)])
lgb_pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp', LGBMClassifier(min_data_in_leaf=40, num_leaves=110))])

In [13]:
lgb_pipeline.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 23160, number of negative: 92643
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7507
[LightGBM] [Info] Number of data points in the train set: 115803, number of used features: 110
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199995 -> initscore=-1.386327
[LightGBM] [Info] Start training from score -1.386327


Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp', LGBMClassifier(min_data_in_leaf=40, num_leaves=110))])

In [17]:
xgb_scaler = StandardScaler()
xgb_smote = SMOTE(sampling_strategy=0.25)
#new_mlp_classifier = MLPClassifier(verbose = 2, hidden_layer_sizes=(16,16),learning_rate_init=0.001, activation="relu", alpha  = 0.0001, max_iter = 15)
xgb_classifier = XGBClassifier(min_child_weight =2,max_depth=15,gamma= 0)
xgb_pipeline = Pipeline([('scaler',xgb_scaler),('smote', xgb_smote),('mlp', xgb_classifier)])
xgb_pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=2, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [18]:
xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=2, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [27]:
nn_scaler = StandardScaler()
nn_smote = SMOTE(sampling_strategy=0.25)
#new_mlp_classifier = MLPClassifier(verbose = 2, hidden_layer_sizes=(16,16),learning_rate_init=0.001, activation="relu", alpha  = 0.0001, max_iter = 15)
nn_classifier = MLPClassifier(learning_rate_init=0.001,hidden_layer_sizes=(16,16),activation="relu",alpha=0.1)
nn_pipeline = Pipeline([('scaler',nn_scaler),('smote', nn_smote),('mlp', nn_classifier)])
nn_pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp', MLPClassifier(alpha=0.1, hidden_layer_sizes=(16, 16)))])

In [28]:
nn_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('mlp', MLPClassifier(alpha=0.1, hidden_layer_sizes=(16, 16)))])

In [54]:
scaler = StandardScaler()
smote = SMOTE(sampling_strategy = 0.25)
#lgb_clf = lgb.LGBMClassifier(num_leaves=110,min_data_in_leaf = 40)
#xgb_clf = XGBClassifier(min_child_weight =2,max_depth=15,gamma= 0)
estimators = [('xgb',xgb_classifier),('nn',nn_classifier)]
combined_clf = StackingClassifier(
estimators=estimators, final_estimator=LogisticRegression(penalty = 'l2'),cv = "prefit")
combined_pipeline = Pipeline([('scaler',scaler),('smote', smote),('clf',combined_clf)])

In [55]:
combined_pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('smote', SMOTE(sampling_strategy=0.25)),
                ('clf',
                 StackingClassifier(cv='prefit',
                                    estimators=[('xgb',
                                                 XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               device=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               fea...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=15,
                                                               max_leaves=None,
                                                               min_child_weight=2,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=None,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...)),
                                                ('nn',
                                                 MLPClassifier(alpha=0.1,
                                                               hidden_layer_sizes=(16,
                                                                                   16)))],
                                    final_estimator=LogisticRegression()))])

## Evaluating on Unseen Test

In [56]:
X_test = test_df[[i for i in X_train]]
y_pred_proba = combined_pipeline.predict_proba(X_test)[:,1]
roc = roc_auc_score(test_df['label'], y_pred_proba)
roc

0.8728232048606657

In [57]:
precision, recall, thresholds = precision_recall_curve(test_df['label'],y_pred_proba)
pr = auc(recall, precision)
pr

0.38991046134924157

In [45]:
0.5*(pr + roc)

0.6553635790954734

In [ ]:
#LGB Pipeline: 0.6564374584801052
#XGB Pipeline: 0.6454325747599323
#NN Pipeline: 0.6598140179425787
#LGB + XGB: 0.6553635790954734 (Best but cannot outperform individual)